In [2]:
import pandas as pd
import numpy as np
import datetime
import sys
import MySQLdb
import matplotlib.pyplot as plt 
import matplotlib.dates as dates
import seaborn as sns
from ipywidgets import interact, fixed, widgets

%matplotlib inline



In [3]:
TICK_INIT = 10*60*60
RESAMP_INIT = 120
RESAMP_MAX = 10000
MAX_DF_LEN = 1
DELTA_SAMP_MIN = 1

In [4]:
def pplot(df, column='none'):
    sns.set_style("white")
    sns.set_style("ticks")
    fig, ax = plt.subplots(figsize=(11, 6))
    
    if isinstance(df, pd.DataFrame):
        rng=df.index
        col=df[column]
        plt.plot(rng, col,marker='.')
        if isinstance(rng[0], datetime.date):
            fig.autofmt_xdate()
            ax.xaxis.set_major_formatter(dates.DateFormatter('%a %m/%d-%H:%M:%S'))
    else:
        plt.plot(df,marker='.')
    
    plt.ylabel(column, fontsize=14)
    plt.tick_params(axis='y', labelsize=12)
    plt.tick_params(which='major', axis='x', labelsize=12)
    plt.grid(axis='y',color='grey', linestyle='--', lw=0.5, alpha=0.5)
    plt.grid(axis='x',color='grey', linestyle='--', lw=0.5, alpha=0.5)
    sns.despine(trim=True, left=True, bottom=True)
    return fig

def pstem(df, column='none'):
    sns.set_style("white")
    sns.set_style("ticks")
    fig, ax = plt.subplots(figsize=(11, 6))
    
    if isinstance(df, pd.DataFrame):
        rng=df.index
        col=df[column]
        plt.stem(rng, col)
        if isinstance(rng[0], datetime.date):
            fig.autofmt_xdate()
            ax.xaxis.set_major_formatter(dates.DateFormatter('%a %m/%d-%H:%M:%S'))
    else:
        plt.stem(df)
    
    plt.ylabel(column, fontsize=14)
    plt.tick_params(axis='y', labelsize=12)
    plt.tick_params(which='major', axis='x', labelsize=12)
    plt.grid(axis='y',color='grey', linestyle='--', lw=0.5, alpha=0.5)
    plt.grid(axis='x',color='grey', linestyle='--', lw=0.5, alpha=0.5)
    sns.despine(trim=True, left=True, bottom=True)
    return fig


In [5]:
def rms(array_like):
    return np.sqrt(np.sum(np.power(array_like,2))/float(len(array_like)))

In [6]:
def slidePlot(dfIn, signal=('atc1_mean',), resample_rate_in_sec=RESAMP_INIT, 
              start=0, end=MAX_DF_LEN, resampMethod='rms'):
    
    sns.set_style("white")
    sns.set_style("ticks")

    dfIn = dfIn.ix[start:end,:]
    col = list(signal)
    title = signal
    dt = str(resample_rate_in_sec) + 'ms'
    
    # get x,y (rng, out) for plot
    if resampMethod == 'rms':
        out = dfIn[col[0]].resample(dt).apply(rms)
    elif resampMethod == 'mean':
        out = dfIn[col[0]].resample(dt).apply(np.mean)
    else:
        out = dfIn[col]
        
    #print out
    #print dfIn[col].resample(dt)
    
    rng = pd.to_datetime(out.index)

    fig, ax = plt.subplots(figsize=(11, 6))
    plt.plot(rng, out, 'v-', lw=2.5, marker='o')

    
    fig.autofmt_xdate()
    ax.xaxis.set_major_formatter(dates.DateFormatter('%a %m/%d-%H:%M:%S'))
    plt.ylabel(col, fontsize=14)
    plt.tick_params(axis='y', labelsize=12)
    plt.tick_params(which='major', axis='x', labelsize=12)
    plt.grid(axis='y',color='grey', linestyle='--', lw=0.5, alpha=0.5)
    plt.grid(axis='x',color='grey', linestyle='--', lw=0.5, alpha=0.5)
    sns.despine(trim=True, left=True, bottom=True)    
    
    return 

def iSlidePlot(df, MAX_RESAMP_TIME=RESAMP_MAX, DELTA_SAMP=DELTA_SAMP_MIN, DELTA_TIME=1):
    
    if df.empty:
        print 'Null DataFrame'
    else:

        %matplotlib inline
    
        attributes = df.columns.tolist()
        MAX_NUM = len(df)-2

        sigWid = widgets.SelectMultiple(
        description = "Signal",
        options=df.columns.tolist(),
        )

        resampMethodWid = widgets.RadioButtons(
        description='Resample Method',
        options=['mean', 'rms', 'none'],
        value='mean',
        )

        resampRateWid = widgets.IntSlider(
        min=DELTA_SAMP,
        max=MAX_RESAMP_TIME,
        step=DELTA_SAMP,
        value=MAX_RESAMP_TIME,
        description='Resample Rate (s)', 
        continuous_update=False,
        )

        startWid = widgets.IntSlider(
        value=0,
        min=0,
        max=len(df),
        step=DELTA_TIME,
        description='Start Sample', 
        continuous_update=False,
        )

        endWid = widgets.IntSlider(
        value=len(df),
        min=0,
        max=len(df),
        step=DELTA_TIME,
        description='End Sample', 
        continuous_update=False,
        )
    
        interact(slidePlot, dfIn=fixed(df), 
             signal=sigWid, 
             resample_rate_in_sec=resampRateWid, 
             start=startWid, 
             end=endWid,
             resampMethod = resampMethodWid)
    return


In [7]:
def yySlidePlot(dfIn, signal1=('atc1_mean'), signal2=('atc2_mean'), resample_rate_in_sec=120, start=0, end=1, resampMethod='rms'):
    
    sns.set_style("white")
    sns.set_style("ticks")
    
    dfIn = dfIn.ix[start:end,:]
    col1 = list(signal1)
    col2 = list(signal2)
    tup1 = ''.join(signal1)
    tup2 = ''.join(signal2)
    dt = str(resample_rate_in_sec) + 's'

    # get x,y (rng, out) for plot
    if resampMethod == 'rms':
        out1 = dfIn[col1].resample(dt).apply(rms)
        out2 = dfIn[col2].resample(dt).apply(rms)
    else:
        out1 = dfIn[col1].resample(dt).mean()
        out2 = dfIn[col2].resample(dt).mean()
    rng = pd.to_datetime(out1.index)

    
    fig, ax = plt.subplots(figsize=[15,12])
    ax2 = ax.twinx()
    ax.plot_date(rng.to_pydatetime(), out1, 'v-', lw=2.5, marker='.', color='c')
    ax2.plot_date(rng.to_pydatetime(), out2, 'v-', lw=2.5, marker='.', color='m')

    fig.autofmt_xdate()
    ax.xaxis.set_major_formatter(dates.DateFormatter('%a %m/%d-%H:%M:%S'))
    ax.set_ylabel(tup1, fontsize=14)    
    ax2.set_ylabel(tup2, fontsize=14)
    plt.tick_params(axis=ax.yaxis, fontsize=28)
    plt.tick_params(which='minor',axis=ax.xaxis, fontsize=28)
    plt.tick_params(which='major', axis=ax.xaxis, fontsize=28)
    plt.tick_params(which='major', axis= ax2, fontsize=28)
    ax.legend(col1, fontsize=18, loc='best')
    ax2.legend(col2, fontsize=18, loc='best')
    
    # just keep one grid (otherwise too confusing)
    ax.grid(axis='y',color='grey', linestyle='--', lw=0.5, alpha=0.5)
    ax.grid(axis='x',color='grey', linestyle='--', lw=0.5, alpha=0.5)
    
    #despine without using sns
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.spines['bottom'].set_visible(False)
    ax2.spines['left'].set_visible(False)
    
    return 

def dualSlidePlot(df, MAX_RESAMP_TIME=120000*30, DELTA_SAMP=1, DELTA_TIME=1):
    
    %matplotlib inline
    
    attributes = df.columns.tolist()
    MAX_NUM = len(df)-2
    
    sigWid1 = widgets.SelectMultiple(
    description = "Signal",
    options=df.columns.tolist()[1:])
    
    sigWid2 = widgets.SelectMultiple(
    description = "Signal",
    options=df.columns.tolist()[1:])
    
    resampMethodWid = widgets.RadioButtons(
    description='Resample Method',
    options=['mean', 'rms'])
    
    resampRateWid = widgets.IntSlider(
    value=1,
    min=DELTA_SAMP,
    max=MAX_RESAMP_TIME,
    step=DELTA_SAMP,
    description='Resample Rate (s)', continuous_update=False)

    startWid = widgets.IntSlider(
    value=0,
    min=0,
    max=len(df),
    step=DELTA_TIME,
    description='Start Sample', continuous_update=False)

    endWid = widgets.IntSlider(
    value=len(df),
    min=0,
    max=len(df),
    step=DELTA_TIME,
    description='End Sample', continuous_update=False)
    
    updateWid = widgets.Checkbox(
    description='Update Data',
    value=False)

    interact(yySlidePlot, dfIn=fixed(df), 
             signal1=sigWid1, signal2=sigWid2, 
             resample_rate_in_sec=resampRateWid, 
             start=startWid, 
             end=endWid,
             resampMethod = resampMethodWid, 
             updateData = updateWid,
             __manual=True)
    return

In [8]:
def jointPlot(df, sig1='pha', sig2='pidtca', deltaTime=5, start=0, end=10):
    temp = df.ix[start:end,:]
    dt = str(deltaTime) + 's'
    a = temp[sig1].resample(dt)
    b = temp[sig2].resample(dt)

#     sns.jointplot(a,b, kind="kde")
#     sns.jointplot(a,b).plot_joint(sns.kdeplot, zorder=0, n_levels=6)
    
    g = sns.jointplot(a,b, kind="kde", size=6).plot_joint(plt.scatter, c='#4CB391')
    plt.xlabel(sig1, fontsize=20)
    plt.ylabel(sig2, fontsize=20)
    #plt.title('Joint Probability Density', fontsize=15 )
    plt.tick_params(axis="both", labelsize=15)
    plt.legend(fontsize=20)

    g.plot_marginals(sns.distplot, hist=True, kde=True)
    g.ax_joint.collections[0].set_alpha(0)
    

    
def iJointPlot(df, MAX_RESAMP_TIME=RESAMP_MAX, DELTA_SAMP=DELTA_SAMP_MIN, DELTA_TIME=1):
    if df.empty:
        print 'Null DataFrame'
    else:

        %matplotlib inline
    
        attributes = df.columns.tolist()
        MAX_NUM = len(df)-2

        sigWid1 = widgets.Select(
        description = "Signal",
        options=df.columns.tolist(),
        #value='pha'
        )

        sigWid2 = widgets.Select(
        description = "Signal",
        options=df.columns.tolist(),
        #value='pidtca'
        )

        resampMethodWid = widgets.RadioButtons(
        description='Resample Method',
        options=['mean', 'rms'],
        value='mean',
        )

        resampRateWid = widgets.IntSlider(
        value=10000,
        min=1,
        max=MAX_RESAMP_TIME,
        step=DELTA_SAMP,
        description='Resample Rate (s)', 
        continuous_update=False,
        )

        startWid = widgets.IntSlider(
        value=0,
        min=0,
        max=len(df),
        step=DELTA_TIME,
        description='Start Sample', 
        continuous_update=False,
        )

        endWid = widgets.IntSlider(
        value=len(df),
        min=0,
        max=len(df),
        step=DELTA_TIME,
        description='End Sample', 
        continuous_update=False,
        )
        
    
        interact(jointPlot, df=fixed(df), 
             sig1=sigWid1,
             sig2=sigWid2,
             deltaTime=resampRateWid, 
             start=startWid, 
             end=endWid, __manual=True)

In [ ]:
#use with plotting plotSensor getter function
def pplotSense(df,sensor, column='none'):
    sns.set_style("white")
    sns.set_style("ticks")
    fig, ax = plt.subplots(figsize=(11, 6))
    
    if isinstance(df, pd.DataFrame):
        rng=df.index
        col=df[column]
        plt.plot(rng, col,marker='.')
        if isinstance(rng[0], datetime.date):
            fig.autofmt_xdate()
            ax.xaxis.set_major_formatter(dates.DateFormatter('%a %m/%d-%H:%M:%S'))
    else:
        plt.plot(df,marker='.')
    
    plt.ylabel('Value', fontsize=14)
    plt.tick_params(axis='y', labelsize=12)
    plt.tick_params(which='major', axis='x', labelsize=12)
    plt.grid(axis='y',color='grey', linestyle='--', lw=0.5, alpha=0.5)
    plt.grid(axis='x',color='grey', linestyle='--', lw=0.5, alpha=0.5)
    sns.despine(trim=True, left=True, bottom=True)
    
    plt.title(sensor, fontsize=20)
    plt.xticks(rotation=45)
    
    return fig
